# Serial Connection


In [387]:
import glob

available_ports = glob.glob("/dev/tty[A-Za-z]*")
print("Available Serial Ports:", available_ports)

Available Serial Ports: ['/dev/ttyUSB0', '/dev/ttyprintk', '/dev/ttyS31', '/dev/ttyS30', '/dev/ttyS29', '/dev/ttyS28', '/dev/ttyS27', '/dev/ttyS26', '/dev/ttyS25', '/dev/ttyS24', '/dev/ttyS23', '/dev/ttyS22', '/dev/ttyS21', '/dev/ttyS20', '/dev/ttyS19', '/dev/ttyS18', '/dev/ttyS17', '/dev/ttyS16', '/dev/ttyS15', '/dev/ttyS14', '/dev/ttyS13', '/dev/ttyS12', '/dev/ttyS11', '/dev/ttyS10', '/dev/ttyS9', '/dev/ttyS8', '/dev/ttyS7', '/dev/ttyS6', '/dev/ttyS5', '/dev/ttyS4', '/dev/ttyS3', '/dev/ttyS2', '/dev/ttyS1', '/dev/ttyS0']


In [388]:
import time, serial, struct
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, clear_output

SERIAL_PORT = "/dev/ttyUSB0"  # Change to match your setup
BAUD_RATE = 115200

In [389]:
def clean_uart():
    with serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=0.5) as ser:
        print(f"Connected to {SERIAL_PORT} @ {BAUD_RATE} baud.")
        ser.reset_input_buffer()
        ser.reset_output_buffer()
        return 1

def read_addr(addr):
    with serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=0.5) as ser:
        print(f"Connected to {SERIAL_PORT} @ {BAUD_RATE} baud.")
        ser.write(bytes([0xA2]))
        rw_addr = addr | 0x80
        ser.write(bytes([rw_addr]))
        return ser.read(2)

def write_addr(addr, data):
    with serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=0.5) as ser:
        print(f"Connected to {SERIAL_PORT} @ {BAUD_RATE} baud.")
        ser.write(bytes([0xA2]))
        ser.write(bytes([addr]))
        ser.write(bytes([data]))
        return ser.read(2)
    
def read_burst():
    with serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=0.5) as ser:
        print(f"Connected to {SERIAL_PORT} @ {BAUD_RATE} baud.")
        ser.write(bytes([0xA3]))
        data = ser.read(8)
        return data

# Suspend mode


Default power mode

Register (0x4B) contains control bits for power control, soft reset and interface SPI mode  
selection. This special control register is accessible in suspend mode.

- 0x4B:
  - 7: soft reset '1'
  - 6: fixed '0'
  - 5: fixed '0'
  - 4: fixed '0'
  - 3: fixed '0'
  - 2: enable SPI3 mode
  - 1: soft reset '1'
  - 0: power control where '0' is suspend mode and '1' is sleep mode

Soft reset is executed when both bits (register 0x4B bit7 and bit1) are set “1”. Soft reset does not
execute a full POR sequence, but all registers are reset except for the “trim” registers above  
register 0x54 and the power control register (0x4B). Soft reset always brings the device into sleep
mode. When device is in the suspend mode, soft reset is ignored and the device remains in  
suspend mode. The two “Soft Reset” bits are reset to “0” automatically after soft reset was  
completed. To perform a full POR reset, bring the device into suspend and then back into sleep
mode.


In [360]:
write_addr(0x4B, 0x01)

Connected to /dev/ttyUSB0 @ 115200 baud.


b'\xff'

In [ ]:
read_addr(0x4B)

Connected to /dev/ttyUSB0 @ 115200 baud.
01: 00000001


# Sleep mode


In this state the user has full access to
the device registers. In particular, the Chip ID can be read. Setting the power control bit to “0”
(register 0x4B bit0) will bring the device back into Suspend mode. From the Sleep mode the user
can put the device back into Suspend mode or into Active mode.

- 0x4C:
  - 7: advanced self-test control indx 1
  - 6: advanced self-test control indx 0
  - 5: data rate control indx 2
  - 4: data rate control indx 1
  - 3: data rate control indx 0
  - 2: operation mode (opmode) control indx 1
  - 1: operation mode (opmode) control indx 0
  - 0: normal self test control

Sleep to active (normal or forced): OpMode bits “00”.
Active to sleep: OpMode bits “11”.


In [407]:
write_addr(0x4C, 0x06)

Connected to /dev/ttyUSB0 @ 115200 baud.


b'\xff'

In [409]:
read_addr(0x4C)

Connected to /dev/ttyUSB0 @ 115200 baud.


b'\xff'

# Active mode


<img src="../assets/bm150//mode_diagram.png">


In this mode the magnetic field measurements are performed and all registers are accessible. In active mode, two operation modes can be distinguished:

- Normal mode:

selected channels are periodically measured according to settings set in
user registers. After measurements are completed, output data is put into data registers
and the device waits for the next measurement period, which is set by programmed output
data rate (ODR).

In principle any desired balance between output noise and
active time (hence power consumption) can be adjusted by the repetition settings for x/y-axis and
z-axis and the output data rate ODR. The average power consumption depends on the ratio of
high current phase time (during data acquisition) and low current phase time (between data  
acquisitions). Hence, the more repetitions are acquired to generate one magnetic field data point,
the longer the active time ratio in one sample phase, and the higher the average current.

With longer internal averaging, the noise level of the output data reduces with increasing number of
repetitions.

- Forced mode (single measurement):

When set by the host, the selected channels are  
measured according to settings programmed in user registers. After measurements are
completed, output data is put into data registers, OpMode register value returns to “11”
and the device returns to sleep mode. The forced mode is useful to achieve synchronized
operation between host microcontroller and BMMdsa150. Also, different data output rates  
from the ones selectable in normal mode can be achieved using forced mode.

By using forced mode, it is possible to trigger new measurements at any rate. The user can  
therefore trigger measurements in a shorter interval than it takes for a measurement cycle to  
complete. If a measurement cycle is not allowed to complete, the resulting data will not be written
into the data registers. To prevent this, the manually triggered measurement intervals must not
be shorter than the active measurement time which is a function of the selected number of  
repetitions. The maximum selectable read-out frequency in forced mode can be calculated as  
follows:

$$
f_{max,ODR} \approx \frac{1}{145 \mu s \times nXY + 500 \mu s \times nZ + 980 \mu s}
$$

Hereby nXY is the number of repetitions on X/Y-axis (not the register value) and nZ the number
of repetitions on Z-axis (not the register value). The performed number of repetitions can be calculated from unsigned register value as:

$$
nXY = 1+2 \times unsigned(REPXY)
$$

- 0x51: contains the number of repetitions for x/y-axis (REPXY).
    $$
    nZ = 1+ unsigned(REPZ)
    $$
- 0x52: contains the number of repetitions for z-axis (REPZ).


In [366]:
def f_max_ODR(nXY, nZ):
    return 1 / (145e-6 * nXY + 500e-6 * nZ + 980e-6)

f_max_ODR(3, 3)

343.0531732418525

Four recommended presets:

- High accuracy
- Enhanced regular
- Regular
- Low power

Reflect the most common usage scenarios, i.e. required output accuracy at a given current consumption.

The four presets consist of the below register configurations, which are automatically set by the
BMM150 API or driver provided by Bosch Sensortec when a preset is selected.


<img src="../assets/bm150//presets.png">

In [367]:
write_addr(0x51, 3) # nXY
write_addr(0x52, 3) # nZ

Connected to /dev/ttyUSB0 @ 115200 baud.
Connected to /dev/ttyUSB0 @ 115200 baud.


b'\xff'

- 0x4C:
  - 7: advanced self-test control indx 1
  - 6: advanced self-test control indx 0
  - 5: data rate control indx 2
  - 4: data rate control indx 1
  - 3: data rate control indx 0
  - 2: operation mode (opmode) control indx 1
  - 1: operation mode (opmode) control indx 0
  - 0: normal self test control


In [368]:
write_addr(0x4c, 0x06)

Connected to /dev/ttyUSB0 @ 115200 baud.


b'\xff'

In [369]:
read_addr(0x4c).hex()

Connected to /dev/ttyUSB0 @ 115200 baud.


'06'

In [370]:
read_addr(0x51)

Connected to /dev/ttyUSB0 @ 115200 baud.


b'\x03'

In [371]:
read_addr(0x52)

Connected to /dev/ttyUSB0 @ 115200 baud.


b'\x03'

# Magnetic field data


Data representation is different between X/Y-axis and Z-axis.

- X- and Y-axis magnetic field data is 13 bits each and stored in two’s complement.

  - DATAX_LSB (0x42) contains 5-bit LSB [4:0] of the 13 bit X-channel.
  - DATAX_MSB (0x43) contains 8-bit MSB [12:5] of the 13 bit X-channel.
  - DATAY_LSB (0x44) contains 5-bit LSB [4:0] of the 13 bit Y-channel.
  - DATAY_MSB (0x45) contains 8-bit MSB [12:5] of the 13 bit Y-channel.

- Z-axis magnetic field data is 15 bit word stored in two’s complement.
  - DATAZ_LSB (0x46) contains 7-bit LSB [6:0] of the 15 bit Z-channel.
  - DATAZ_MSB (0x47) contains 8-bit MSB [14:7] of the 15 bit Z-channel.

Temperature compensation on the host is used to get ideally matching sensitivity  
over the full temperature range. The temperature compensation is based on a resistance  
measurement of the hall sensor plate. The resistance value is represented by a 14 bit unsigned
output word:

- RHALL_LSB (0x48) contains 6-bit LSB [5:0] of the 14 bit RHALL-channel.
- RHALL_MSB (0x49) contains 8-bit MSB [13:6] of the 14 bit RHALL-channel.

After all enabled axes have been measured; complete data packages consisting of DATAX,  
DATAY, DATAZ and RHALL are updated at once in the data registers. This way, it is prevented
that a following axis is updated while the first axis is still being read (axis mix-up) or that MSB part
of an axis is updated while LSB part is being read.

While reading from any data register, data register update is blocked. Instead, incoming new data
is written into shadow registers which will be written to data registers after the previous read  
sequence is completed (i.e. upon stop condition in I²C mode, or CSB going high in SPI mode,
respectively). Hence, it is recommended to read out at all data at once (0x42 to 0x49 or 0x4A if
status bits are also required) with a burst read. Single bytes or axes can be read out, while in 
this case it is not assured that adjacent registers are not updated during readout sequence.

The  “Data  ready  status”  bit  (register  0x48  bit0)  is  set  “1”  when  the  data  registers  have  been  
updated but the data was not yet read out over digital interface. Data ready is cleared (set “0”) 
directly after completed read out of any of the data registers and subsequent stop condition (I²C) 
or lifting of CSB (SPI).

In  addition,  when  enabled  the  “Data  overrun”  bit  (register  0x4A  bit7)  turns  “1”  whenever  data  
registers are updated internally, but the old data was not yet read out over digital interface (i.e. 
data ready bit was still high). The “Data overrun” bit is cleared when the interrupt status register 
0x4A is read out. This function needs to be enabled separately by setting the “Data overrun En” 
bit (register 0x4D bit7).

<img src="../assets/bm150//drdy.png">

When the “Data Ready Pin En” bit (register 0x4E bit7) is set, the Data Ready (DRDY) interrupt 
event is flagged on the BMM150’s DRDY output pin (by default the “Data Ready Pin En” bit is not 
set and DRDY pin is in high-Z state).

The interrupt status registers are updated together with writing new data into the magnetic field 
data registers. The status bits for Low-/High-Threshold interrupts are located in register 0x4A, the 
Data Ready (DRDY) status flag is located at register 0x48 bit0.

This interrupt serves for synchronous reading of magnetometer data. It is generated after storing 
a new set of values (DATAX, DATAY, DATAZ, RHALL) in the data registers

It is enabled (disabled) by writing “1” (“0”) to “Data Ready pin En” in register 0x4E bit7.

- 0x4E: contains control bits interrupt settings and axes enable bits.
  - 7: enables DRDY (default is disabled)
  - 6: enables INT "interrupt pin en" (default is '0')
  - 5: enables channel Z and resistance measurement (default is '0')
  - 4: enables channel Y and resistance measurement (default is '0')
  - 3: enables channel X and resistance measurement (default is '0')
  - 2: DRDY polarity (default is '1')
    - '0': active low
    - '1': active high
  - 1: INT latching (default is '1')
    - '0': non-latched (interrupt pin is on as long as the condition is fullfilled)
    - '1': latched (interrupt pin is on until interrupt status register 0x4A is read)
  - 0: INT polarity (default is '1')
    - '0': active low
    - '1': active high

In [372]:
write_addr(0x4E, 0xBF)

Connected to /dev/ttyUSB0 @ 115200 baud.


b'\xff'

In [373]:
read_addr(0x4E)

Connected to /dev/ttyUSB0 @ 115200 baud.


b'\xbf'

In [ ]:
def twos_complement(value, bits):
    """Convert an integer value to signed two's complement."""
    if value & (1 << (bits - 1)):
        value -= 1 << bits
    return value

def parse_burst(raw):
    # --- X-axis: 13 bits, two's complement
    x_raw = ((raw[1] << 5) | (raw[0] & 0x1F))
    x_val = twos_complement(x_raw, 13)
    
    # --- Y-axis: 13 bits, two's complement
    y_raw = ((raw[3] << 5) | (raw[2] & 0x1F))
    y_val = twos_complement(y_raw, 13)
    
    # --- Z-axis: 15 bits, two's complement
    z_raw = ((raw[5] << 7) | (raw[4] & 0x7F))
    z_val = twos_complement(z_raw, 15)
    
    # --- RHALL: 14 bits unsigned
    rhall_raw = ((raw[7] << 6) | (raw[6] & 0x3F))
    
    # --- Data Ready bit (bit0 of register 0x48)
    data_ready = raw[6] & 0x01
    
    return {
        "X": x_val,
        "Y": y_val,
        "Z": z_val,
        "RHALL": rhall_raw,
        "DataReady": bool(data_ready)
    }

In [401]:
clean_uart()
write_addr(0x4C, 0x00)  # Active


burst_data = read_burst()
results = parse_burst(burst_data)


write_addr(0x4C, 0x06)  # Sleep

Connected to /dev/ttyUSB0 @ 115200 baud.
Connected to /dev/ttyUSB0 @ 115200 baud.
Connected to /dev/ttyUSB0 @ 115200 baud.
Connected to /dev/ttyUSB0 @ 115200 baud.


b'\xff'